<a href="https://colab.research.google.com/github/Larbi96-enet/Cure-EMG-signal-classification/blob/main/processdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # charts
import scipy as sp
from scipy.fft import fft, ifft
from scipy import signal
import csv
from numpy import savetxt
import os
import glob


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def remove_mean(emg,time):
    # process EMG signal: remove mean
    emg_correctmean = emg - np.mean(emg)
    return emg_correctmean



In [ ]:
def emg_filter(emg_correctmean,time):

    # create bandpass filter for EMG
    high = 20 / (1000 / 2)
    low = 450 / (1000 / 2)
    b, a = sp.signal.butter(4, [high, low], btype='bandpass')

    # process EMG signal: filter EMG
    emg_filtered = sp.signal.filtfilt(b, a, emg_correctmean)
    return emg_filtered


In [ ]:
def emg_rectifier(emg_filtered,time):
    # process EMG signal: rectify
    emg_rectified = abs(emg_filtered)
    return emg_rectified


In [ ]:
def concatination(indir,outfile):
    os.chdir(indir)
    file_list = glob.glob("*.csv")
    dflist =[]
    for csv_file in file_list:
        df=pd.read_csv(csv_file)
        dflist.append(df)
        P_EMG = pd.concat(dflist,axis=1)
        P_EMG.to_csv(outfile,index=None)


In [ ]:
def stepwise_action_window_creator(df,window_size,step_size):
    dflist = []
    index = df.index.values
    for i in range(0,len(index),step_size):
        if(i > len(index) - window_size):
            break;
        else:
            dflist.append(df.iloc[i:i+window_size])
    for i in range(0,len(dflist)):
        index = dflist[i].index.values
        dflist[i] = dflist[i].assign(time = range(index[-1]+1 - index[0]))
        dflist[i] = dflist[i].assign(act_id = [i]*len(dflist[i]))
    result = pd.concat(dflist)
    return result


In [ ]:
def zero_crossing(temp,threshold):
    zc = 0
    for i in range(len(temp)-1):
        if((temp[i]*temp[i+1]) < 0 and abs(temp[i]-temp[i+1]) >= threshold):
            zc += 1
    return zc


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/data/EMG-data.csv')
Inputs_signals = data.drop(['time','label','class'],axis=1)
time=data.time
columns_ls=Inputs_signals.columns
Emg_columns=[]
processedEMG = pd.DataFrame()


In [ ]:
for i in Inputs_signals:
    j= list(columns_ls).index(i)
    raw_emg = Inputs_signals[i]
    emg_correctmean=remove_mean(raw_emg.values,time)
    emg_filtered= emg_filter(emg_correctmean,time)
    emg_rectified=emg_rectifier(emg_filtered,time)
    column_name = "channel{}".format(j)
    file_name = "processedEMG{}.csv".format(j)
    with open(file_name, 'w', newline='') as file:
        savetxt(file_name, emg_rectified, delimiter=',', header=column_name)
